In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="dq96YdSMsS3dS5azjTCD")
project = rf.workspace("object-detection-onsq3").project("traffic-light-tvvnr")
dataset = project.version(1).download("yolov5")


In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

In [4]:
%cat /content/Traffic-light-1/data.yaml

names:
- green
- red
- yellow
nc: 3
roboflow:
  license: CC BY 4.0
  project: traffic-light-tvvnr
  url: https://universe.roboflow.com/object-detection-onsq3/traffic-light-tvvnr/dataset/1
  version: 1
  workspace: object-detection-onsq3
test: ../test/images
train: Traffic-light-1/train/images
val: Traffic-light-1/valid/images


In [5]:
%cd /
from glob import glob

train_img_list = glob('/content/Traffic-light-1/train/images/*.jpg')

print(len(train_img_list))

/
261


In [6]:
%cd /
from glob import glob

val_img_list = glob('/content/Traffic-light-1/test/images/*.jpg')

print(len(val_img_list))

/
75


In [7]:
with open('/content/Traffic-light-1/train.txt','w') as f :
  f.write('\n'.join(train_img_list)+'\n')

with open('/content/Traffic-light-1/val.txt','w') as f :
  f.write('\n'.join(val_img_list)+'\n')

In [8]:
import yaml

with open('/content/Traffic-light-1/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/content/Traffic-light-1/train.txt'
data['val'] = '/content/Traffic-light-1/val.txt'

with open('/content/Traffic-light-1/data.yaml','w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['green', 'red', 'yellow'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'traffic-light-tvvnr', 'url': 'https://universe.roboflow.com/object-detection-onsq3/traffic-light-tvvnr/dataset/1', 'version': 1, 'workspace': 'object-detection-onsq3'}, 'test': '../test/images', 'train': 'Traffic-light-1/train/images', 'val': 'Traffic-light-1/valid/images'}
{'names': ['green', 'red', 'yellow'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'traffic-light-tvvnr', 'url': 'https://universe.roboflow.com/object-detection-onsq3/traffic-light-tvvnr/dataset/1', 'version': 1, 'workspace': 'object-detection-onsq3'}, 'test': '../test/images', 'train': '/content/Traffic-light-1/train.txt', 'val': '/content/Traffic-light-1/val.txt'}


In [9]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/Traffic-light-1/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name light

/content/yolov5
2023-11-26 18:34:54.412803: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 18:34:54.412858: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 18:34:54.412893: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/Traffic-light-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_

In [10]:
from IPython.display import Image
import os

val_img_path = "/content/횡단보도_cross_car.png"
print("이미지 경로:", val_img_path)

# YOLOv5 감지 실행
!python detect.py --weights /content/yolov5/runs/train/light/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}"

# 디버깅을 위해 출력 디렉토리 출력
output_path = os.path.join('/content/yolov5/inference/output', os.path.basename(val_img_path))

이미지 경로: /content/횡단보도_cross_car.png
detect: weights=['/content/yolov5/runs/train/light/weights/best.pt'], source=/content/횡단보도_cross_car.png, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/횡단보도_cross_car.png: 224x416 (no detections), 42.5ms
Speed: 0.3ms pre-process, 42.5ms inference, 0.9ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp
